In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from scipy.sparse import csr_matrix
import random as rnd
import pickle
import sys


from datetime import datetime


from sklearn import preprocessing
from sklearn.model_selection import train_test_split




In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
def generate_random_array(median,std,size):
    output=[0]*size    
    for index in range(0,size//2):
        random_nr=rnd.uniform(-std,std)
        output[2*index]=median+random_nr
        output[2*index+1]=median-random_nr
    if(size % 2 != 0):
        output[size-1]=rnd.uniform(median-std,median+std)
    return output



In [3]:
def label_attacks(x):
  if x != 'BENIGN' and x != 'DDoS' and x != 'PortScan':
    return 'DoS'
  return x

In [4]:
def is_attacking(g):
  if len(set(g[' Label'])) == 1 and list(g[' Label'])[0] == 0:
    return False
  return True

In [6]:
traffic = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TrafficLabelling/combined.csv")
print(f'Number of Rows: {traffic.shape[0]}')
print(f'Number of Columns: {traffic.shape[1]}')
pd.set_option('display.max_colwidth', None)
traffic.head()

Number of Rows: 96448
Number of Columns: 84


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,10.0.0.1-10.0.0.2-3451-8996-6,10.0.0.1,3451,10.0.0.2,8996,6,10/02/2021 01:01:25 pm,501082,2,0,...,20,0.0,0.0,0.0,0.0,1.612924e+15,0.0,1.612924e+15,1.612924e+15,NeedManualLabel
1,10.0.0.1-10.0.0.2-3451-8998-6,10.0.0.1,3451,10.0.0.2,8998,6,10/02/2021 01:01:26 pm,500525,2,0,...,20,0.0,0.0,0.0,0.0,1.612924e+15,0.0,1.612924e+15,1.612924e+15,NeedManualLabel
2,10.0.0.1-10.0.0.2-3451-9000-6,10.0.0.1,3451,10.0.0.2,9000,6,10/02/2021 01:01:27 pm,501031,2,0,...,20,0.0,0.0,0.0,0.0,1.612924e+15,0.0,1.612924e+15,1.612924e+15,NeedManualLabel
3,10.0.0.1-10.0.0.2-3451-9002-6,10.0.0.1,3451,10.0.0.2,9002,6,10/02/2021 01:01:28 pm,500477,2,0,...,20,0.0,0.0,0.0,0.0,1.612924e+15,0.0,1.612924e+15,1.612924e+15,NeedManualLabel
4,10.0.0.1-10.0.0.2-3451-9004-6,10.0.0.1,3451,10.0.0.2,9004,6,10/02/2021 01:01:29 pm,501388,2,0,...,20,0.0,0.0,0.0,0.0,1.612924e+15,0.0,1.612924e+15,1.612924e+15,NeedManualLabel


In [8]:
# Remove duplicate entries
traffic.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

# reformat Timestamp to Unix time
# traffic[' Timestamp'] = pd.to_datetime(traffic[' Timestamp'])
# traffic[' Timestamp']= traffic[' Timestamp'].values.astype(np.int64) // 10 ** 9

# remove Timestamp column
traffic = traffic.drop(columns=['Timestamp'])

# Remove constant columns
traffic = traffic.loc[:, traffic.apply(pd.Series.nunique) != 1]




In [9]:
# Remove column with NaN or Inf
traffic = traffic[~traffic.isin([np.nan, np.inf, -np.inf]).any(1)]

# remove Flow ID
traffic = traffic.drop(columns=['Flow ID'])



3

In [10]:
raw_labels = set(list(traffic[' Label']))

traffic = traffic[traffic[' Label']  != "SSH-Patator"]
traffic = traffic[traffic[' Label']  != "Heartbleed"]
traffic = traffic[traffic[' Label']  != "FTP-Patator"]
traffic = traffic[traffic[' Label']  != "Infiltration"]
traffic = traffic[traffic[' Label']  != "Bot"]
raw_labels

In [10]:

traffic[' Label'] = traffic[' Label'].apply(label_attacks)


raw_labels = list(traffic[' Label'])

for raw_label in set(raw_labels):
  print(str(raw_label) + ": " + str(raw_labels.count(raw_label)))

traffic[' Label'] = traffic[' Label'].astype('category')
traffic[' Label'] = traffic[' Label'].cat.codes



DDoS: 128025
PortScan: 158804
BENIGN: 2103072
DoS: 251711


In [11]:
print("BENIGN traffic /0: ", list(traffic[' Label']).count(0))
print("DDoS traffic /1: ", list(traffic[' Label']).count(1))
print("DoS traffic /2: ", list(traffic[' Label']).count(2))
print("PortScan traffic /3: ", list(traffic[' Label']).count(3))


BENIGN traffic /0:  2103072
DDoS traffic /1:  128025
DoS traffic /2:  251711
PortScan traffic /3:  158804


In [12]:
columns = traffic.columns

In [13]:
# combine ' Source IP' and ' Source Port'
traffic[' Source IP'] = traffic[' Source IP'] + ':' + traffic[' Source Port'].astype(str)
traffic[' Destination IP'] = traffic[' Destination IP'] + ':' + traffic[' Destination Port'].astype(str)
traffic.rename({' Source IP': ' SrcHost', ' Destination IP': ' DstHost'}, axis=1, inplace=True)

traffic = traffic.drop(columns=[' Source Port'])
traffic = traffic.drop(columns=[' Destination Port'])

traffic

,SrcHost,DstHost,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,104.16.207.165:443,192.168.10.5:54865,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,9.000000,6.0,0.0,40,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,104.16.28.216:80,192.168.10.5:55054,6,109,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,1.100917e+05,18348.623850,109.0,0.000000,109.0,109.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,9174.311927,9174.311927,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,104.16.28.216:80,192.168.10.5:55055,6,52,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,2.307692e+05,38461.538460,52.0,0.000000,52.0,52.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,19230.769230,19230.769230,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,104.17.241.25:443,192.168.10.16:46236,6,34,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,3.529412e+05,58823.529410,34.0,0.000000,34.0,34.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,29411.764710,29411.764710,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,104.19.196.102:443,192.168.10.5:54863,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,9.000000,6.0,0.0,40,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2660372,192.168.10.14:51114,192.168.10.3:53,17,32215,4,2,112.0,152.0,28.0,28.0,28.0,0.00000,76.0,76.0,76.0,0.0,8.194940e+03,186.248642,6443.0,13617.579480,30780.0,3.0,30832.0,10277.33333,17755.84381,30780.0,4.0,3.0,3.0,0.0,3.0,3.0,0,0,80,64,124.165761,62.082881,28.0,76.0,41.714286,23.421602,548.571429,0,0,0,0,0,0,0,0,0.0,48.666667,28.0,76.0,80,4,112,2,152,-1,-1,3,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2660373,192.168.10.16:24054,192.168.10.3:53,17,324,2,2,84.0,362.0,42.0,42.0,42.0,0.00000,181.0,181.0,181.0,0.0,1.376543e+06,12345.679010,108.0,183.597386,320.0,2.0,2.0,2.00000,0.00000,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0,0,40,40,6172.839506,6172.839506,42.0,181.0,97.600000,76.133435,5796.300000,0,0,0,0,0,0,0,0,1.0,122.000000,42.0,181.0,40,2,84,2,362,-1,-1,1,20,0.0,

In [14]:
### drop benign traffics from hosts that send out attacking traffics

# 1. find out host with attacking + benign traffics
atker_benign = traffic[traffic.groupby([' SrcHost'])[' Label'].transform('nunique') > 1]
# 2. select benign traffic from those host
atker_benign = atker_benign[atker_benign[' Label'] == 0]
# 3. remove benign
traffic = traffic.drop(atker_benign.index)



traffic


,SrcHost,DstHost,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,104.16.207.165:443,192.168.10.5:54865,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,9.000000,6.0,0.0,40,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,104.16.28.216:80,192.168.10.5:55054,6,109,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,1.100917e+05,18348.623850,109.0,0.000000,109.0,109.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,9174.311927,9174.311927,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,104.16.28.216:80,192.168.10.5:55055,6,52,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,2.307692e+05,38461.538460,52.0,0.000000,52.0,52.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,19230.769230,19230.769230,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,104.17.241.25:443,192.168.10.16:46236,6,34,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,3.529412e+05,58823.529410,34.0,0.000000,34.0,34.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,29411.764710,29411.764710,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,104.19.196.102:443,192.168.10.5:54863,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,9.000000,6.0,0.0,40,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2660372,192.168.10.14:51114,192.168.10.3:53,17,32215,4,2,112.0,152.0,28.0,28.0,28.0,0.00000,76.0,76.0,76.0,0.0,8.194940e+03,186.248642,6443.0,13617.579480,30780.0,3.0,30832.0,10277.33333,17755.84381,30780.0,4.0,3.0,3.0,0.0,3.0,3.0,0,0,80,64,124.165761,62.082881,28.0,76.0,41.714286,23.421602,548.571429,0,0,0,0,0,0,0,0,0.0,48.666667,28.0,76.0,80,4,112,2,152,-1,-1,3,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2660373,192.168.10.16:24054,192.168.10.3:53,17,324,2,2,84.0,362.0,42.0,42.0,42.0,0.00000,181.0,181.0,181.0,0.0,1.376543e+06,12345.679010,108.0,183.597386,320.0,2.0,2.0,2.00000,0.00000,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0,0,40,40,6172.839506,6172.839506,42.0,181.0,97.600000,76.133435,5796.300000,0,0,0,0,0,0,0,0,1.0,122.000000,42.0,181.0,40,2,84,2,362,-1,-1,1,20,0.0,

In [15]:
print("BENIGN traffic /0: ", list(traffic[' Label']).count(0))
print("DDoS traffic /1: ", list(traffic[' Label']).count(1))
print("DoS traffic /2: ", list(traffic[' Label']).count(2))
print("PortScan traffic /3: ", list(traffic[' Label']).count(3))


BENIGN traffic /0:  2063624
DDoS traffic /1:  128025
DoS traffic /2:  251711
PortScan traffic /3:  158804


In [16]:
### drop dos or/and ddos traffic of hosts that sends out multiple attacks type traffic
# 1. host with multiple attack types
multiple_types = traffic[traffic.groupby([' SrcHost'])[' Label'].transform('nunique') > 1]

# 2. select only ddos and dos traffic
multiple_types = multiple_types[multiple_types[' Label'] != 3]

# 3. remove ddos and dos traffic
traffic = traffic.drop(multiple_types.index)



In [17]:
print("BENIGN traffic /0: ", list(traffic[' Label']).count(0))
print("DDoS traffic /1: ", list(traffic[' Label']).count(1))
print("DoS traffic /2: ", list(traffic[' Label']).count(2))
print("PortScan traffic /3: ", list(traffic[' Label']).count(3))


BENIGN traffic /0:  2063624
DDoS traffic /1:  90692
DoS traffic /2:  195
PortScan traffic /3:  158804


In [18]:
traffic

,SrcHost,DstHost,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,104.16.207.165:443,192.168.10.5:54865,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,9.000000,6.0,0.0,40,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,104.16.28.216:80,192.168.10.5:55054,6,109,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,1.100917e+05,18348.623850,109.0,0.000000,109.0,109.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,9174.311927,9174.311927,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,104.16.28.216:80,192.168.10.5:55055,6,52,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,2.307692e+05,38461.538460,52.0,0.000000,52.0,52.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,19230.769230,19230.769230,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,104.17.241.25:443,192.168.10.16:46236,6,34,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,3.529412e+05,58823.529410,34.0,0.000000,34.0,34.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,29411.764710,29411.764710,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,104.19.196.102:443,192.168.10.5:54863,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,9.000000,6.0,0.0,40,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2660372,192.168.10.14:51114,192.168.10.3:53,17,32215,4,2,112.0,152.0,28.0,28.0,28.0,0.00000,76.0,76.0,76.0,0.0,8.194940e+03,186.248642,6443.0,13617.579480,30780.0,3.0,30832.0,10277.33333,17755.84381,30780.0,4.0,3.0,3.0,0.0,3.0,3.0,0,0,80,64,124.165761,62.082881,28.0,76.0,41.714286,23.421602,548.571429,0,0,0,0,0,0,0,0,0.0,48.666667,28.0,76.0,80,4,112,2,152,-1,-1,3,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2660373,192.168.10.16:24054,192.168.10.3:53,17,324,2,2,84.0,362.0,42.0,42.0,42.0,0.00000,181.0,181.0,181.0,0.0,1.376543e+06,12345.679010,108.0,183.597386,320.0,2.0,2.0,2.00000,0.00000,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0,0,40,40,6172.839506,6172.839506,42.0,181.0,97.600000,76.133435,5796.300000,0,0,0,0,0,0,0,0,1.0,122.000000,42.0,181.0,40,2,84,2,362,-1,-1,1,20,0.0,

In [19]:
columns = list(traffic.columns)


In [20]:
# create host dict {host: node_id}
src_hosts = set(traffic[' SrcHost'])
dst_hosts = set(traffic[' DstHost'])
hosts = list(src_hosts.union(dst_hosts))

                
host_dict = {hosts[i]: i for i in range(len(hosts))}   # dictionary of ip addresses { ip_addr: node_id }
print("Number of nodes: " + str(len(host_dict)))

Number of nodes: 435670


In [21]:
# adding a group_size column to the original dataframe
traffic['group_size'] = traffic.groupby([' SrcHost'])[' DstHost'].transform('size')


In [22]:
# define aggregation functions for all columns
aggregations = {col_name: "mean" for col_name in columns}
aggregations.pop(' Protocol', None)
aggregations.pop(' SrcHost', None)
aggregations.pop(' DstHost', None)
aggregations.pop(' Label', None)

aggregations.update({' Label': lambda x : list(set(x))[0]})
print(aggregations)


{' Flow Duration': 'mean', ' Total Fwd Packets': 'mean', ' Total Backward Packets': 'mean', 'Total Length of Fwd Packets': 'mean', ' Total Length of Bwd Packets': 'mean', ' Fwd Packet Length Max': 'mean', ' Fwd Packet Length Min': 'mean', ' Fwd Packet Length Mean': 'mean', ' Fwd Packet Length Std': 'mean', 'Bwd Packet Length Max': 'mean', ' Bwd Packet Length Min': 'mean', ' Bwd Packet Length Mean': 'mean', ' Bwd Packet Length Std': 'mean', 'Flow Bytes/s': 'mean', ' Flow Packets/s': 'mean', ' Flow IAT Mean': 'mean', ' Flow IAT Std': 'mean', ' Flow IAT Max': 'mean', ' Flow IAT Min': 'mean', 'Fwd IAT Total': 'mean', ' Fwd IAT Mean': 'mean', ' Fwd IAT Std': 'mean', ' Fwd IAT Max': 'mean', ' Fwd IAT Min': 'mean', 'Bwd IAT Total': 'mean', ' Bwd IAT Mean': 'mean', ' Bwd IAT Std': 'mean', ' Bwd IAT Max': 'mean', ' Bwd IAT Min': 'mean', 'Fwd PSH Flags': 'mean', ' Fwd URG Flags': 'mean', ' Fwd Header Length': 'mean', ' Bwd Header Length': 'mean', 'Fwd Packets/s': 'mean', ' Bwd Packets/s': 'mean'

In [23]:
# low activity grouping
low_activities = traffic[traffic['group_size'] == 1]
low_activities = low_activities[low_activities[' Label'] == 0]
low_activities = low_activities.groupby(' SrcHost', as_index=False).agg(aggregations)
low_activities



,SrcHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.0.0.0:0,36685713,2,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.054517,36700000.0,0.000000,36700000.0,36700000.0,36700000.0,36700000.0,0.000000,36700000.0,36700000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.054517,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,0
1,1.193.219.21:443,648,3,0,258.0,0.0,258.0,0.0,86.0,148.956369,0.0,0.0,0.0,0.0,3.981481e+05,4629.629630,324.0,452.548340,644.0,4.0,648.0,324.0,452.548340,644.0,4.0,0.0,0.0,0.0,0.0,0.0,0,0,96,0,4629.629630,0.000000,0.0,258.0,64.5,129.000000,16641.0,0,0,0,0,1,0,0,0,0.0,86.0,86.0,0.0,96,3,258,0,0,61,-1,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.234.58.131:80,186262,2,0,6.0,0.0,6.0,0.0,3.0,4.242641,0.0,0.0,0.0,0.0,3.221269e+01,10.737563,186262.0,0.000000,186262.0,186262.0,186262.0,186262.0,0.000000,186262.0,186262.0,0.0,0.0,0.0,0.0,0.0,0,0,52,0,10.737563,0.000000,0.0,6.0,2.0,3.464102,12.0,0,0,0,0,1,0,0,0,0.0,3.0,3.0,0.0,52,2,6,0,0,140,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.9.56.32:80,83,1,1,6.0,6.0,6.0,6.0,6.0,0.000000,6.0,6.0,6.0,0.0,1.445783e+05,24096.385542,83.0,0.000000,83.0,83.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,12048.192771,12048.192771,6.0,6.0,6.0,0.000000,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,980,253,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,101.102.235.200:443,2,1,1,6.0,6.0,6.0,6.0,6.0,0.000000,6.0,6.0,6.0,0.0,6.000000e+06,1000000.000000,2.0,0.000000,2.0,2.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,500000.000000,500000.000000,6.0,6.0,6.0,0.000000,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,279,251,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121451,98.158.54.101:443,67580,2,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,29.594555,67580.0,0.000000,67580.0,67580.0,67580.0,67580.0,0.000000,67580.0,67580.0,0.0,0.0,0.0,0.0,0.0,0,0,64,0,29.594555,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,64,2,0,0,0,62512,-1,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
121452,98.158.54.103:443,119023,2,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,16.803475,119023.0,0.000000,119023.0,119023.0,119023.0,119023.0,0.000000,119023.0,119023.0,0.0,0.0,0.0,0.0,0.0,0,0,64,0,16.803475,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,64,2,0,0,0,62512,-1,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
121453,98.158.54.104:443,2110,3,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,1421.800948,1055.0,1429.769912,2066.0,44.0,2110.0,1055.0,1429.769912,2066.0,44.0,0.0,0.0,0.0,

In [24]:
# dos grouping
dos_activities = traffic[traffic[' Label'] == 2]
dos_activities = dos_activities.groupby(' SrcHost', as_index=False).agg(aggregations)




In [25]:
basic_flows = traffic.groupby([' SrcHost', ' DstHost'], as_index=False)

basic_flows = basic_flows.agg(aggregations)
basic_flows



,SrcHost,DstHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.0.0.0:0,224.0.0.22:0,36685713.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.054517,36700000.0,0.0000,36700000.0,36700000.0,36700000.0,36700000.0,0.0,36700000.0,36700000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054517,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,0
1,1.1.70.73:80,192.168.10.15:52772,344.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,34883.720930,5813.953488,344.0,0.0000,344.0,344.0,344.0,344.0,0.0,344.0,344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,5813.953488,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,2.0,12.0,0.0,0.0,4380.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.1.70.73:80,192.168.10.17:48301,315.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,6349.206349,315.0,0.0000,315.0,315.0,315.0,315.0,0.0,315.0,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,6349.206349,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.1.70.73:80,192.168.10.19:35264,179391.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,16.723247,89695.5,126775.0535,179339.0,52.0,179391.0,179391.0,0.0,179391.0,179391.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,32.0,11.148831,5.574416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,1.0,0.0,4677.0,240.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.1.70.73:80,192.168.10.19:35275,417.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4796.163070,417.0,0.0000,417.0,417.0,417.0,417.0,0.0,417.0,417.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,4796.163070,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335880,99.192.248.32:80,192.168.10.5:49372,55.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,218181.818182,36363.636364,55.0,0.0000,55.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,18181.818182,18181.818182,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,1.0,6.0,1.0,6.0,46.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1335881,99.192.248.32:80,192.168.10.5:49419,45.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,266666.666667,44444.444444,45.0,0.0000,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,22222.222222,22222.222222,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.

In [26]:
#activity_flows
activity_flows = basic_flows.groupby([' SrcHost'], as_index=False)
activity_flows = activity_flows.agg(aggregations)

print(set(activity_flows[' Label']))
non_atkers = activity_flows.groupby([' Label']).get_group(0)
ddos_atkers = activity_flows.groupby([' Label']).get_group(1)
dos_atkers = activity_flows.groupby([' Label']).get_group(2)
portscan_atkers = activity_flows.groupby([' Label']).get_group(3)

print(activity_flows)
print("Number of non-atkers: " + str(len(non_atkers)))
print("Number of ddos " + str(len(ddos_atkers)))
print("Number of dos " + str(len(dos_atkers)))
print("Number of portscan " + str(len(portscan_atkers)))


{0, 1, 2, 3}
                  SrcHost   Flow Duration  ...    Idle Min   Label
0               0.0.0.0:0    3.668571e+07  ...  36700000.0       0
1            1.1.70.73:80    4.511675e+04  ...         0.0       0
2        1.193.219.21:443    6.480000e+02  ...         0.0       0
3        1.193.219.24:443    1.038322e+05  ...         0.0       0
4          1.226.51.14:80    4.217540e+04  ...         0.0       0
...                   ...             ...  ...         ...     ...
425087  98.158.58.110:443    4.627100e+04  ...         0.0       0
425088  98.158.62.102:443    4.732533e+04  ...         0.0       0
425089  98.158.62.103:443    1.156000e+03  ...         0.0       0
425090   99.192.248.32:80    8.496417e+03  ...         0.0       0
425091  99.193.229.25:443    3.000000e+00  ...         0.0       0

[425092 rows x 71 columns]
Number of non-atkers: 376722
Number of ddos 34216
Number of dos 12
Number of portscan 14142


In [27]:
# create low profiles dataframe
num_hosts = len(host_dict)
num_src_hosts = len(activity_flows[' SrcHost'])
num_noflow_hosts = num_hosts - num_src_hosts


no_flow_prof_columns = columns[3:-2]
print(no_flow_prof_columns)
num_low_hosts = round(0.4*num_noflow_hosts)   #### adjust benign/attackers ratio here
low_activities_prof = np.zeros((num_low_hosts, len(no_flow_prof_columns)))
for col, attr in enumerate(no_flow_prof_columns):
  median = low_activities[attr].median()
  std = np.std(low_activities[attr])
  low_activities_prof[:, col] = generate_random_array(median, std, num_low_hosts)


low_profile_df = pd.DataFrame(low_activities_prof, columns = no_flow_prof_columns)
low_profile_df[' Label'] = [0 for i in range(num_low_hosts)]
low_profile_df[' SrcHost'] = [hosts[num_src_hosts + i] for i in range(num_low_hosts)]
low_profile_df




[' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Fwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' ECE Flag Count', ' Down/Up R

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Label,SrcHost
0,-1.717304e+06,0.538831,28.290366,-1811.743072,18744.836759,167.914581,-6.573419,19.299274,-18.945228,-364.811214,1.853844,99.803761,13.388059,4.082302e+06,-81660.147237,-522536.422049,5.642669e+05,4.673460e+05,-68737.375930,1.876192e+06,7.586381e+05,-1.958049e+05,3.915408e+06,122583.614613,3.633214e+06,-1.303403e+06,1.529619e+06,1.438183e+06,-2062.376154,-0.055269,0.011951,1.711448e+06,-9.468616e+04,-24814.903310,-33844.194361,54.613805,633.133939,42.735361,117.207321,-13839.092811,-0.053900,-0.005219,-0.001761,-0.291046,0.128461,-0.144868,-0.005487,0.046597,1.157210,-51.091316,-47.756975,213.903218,-1.823046e+07,2.495813,6591.494286,23.591188,-13096.294371,4854.242874,296.773863,0.082365,-1.483011e+06,-386778.797645,158444.571203,318998.318589,-327478.592132,-1.316769e+06,-580084.265339,1.309700e+06,0,192.168.10.50:4444
1,1.765224e+06,3.461169,-24.290366,1955.743072,-18352.836759,-95.914581,72.573419,50.700726,18.945228,562.811214,164.146156,94.196239,-13.388059,-4.053993e+06,81994.679310,538459.755383,-5.368151e+05,-4.195260e+05,68743.375930,-1.876186e+06,-7.586321e+05,1.958049e+05,-3.915402e+06,-122577.614613,-3.633206e+06,1.303411e+06,-1.529619e+06,-1.438175e+06,2070.376154,0.055269,-0.011951,-1.711320e+06,9.481416e+04,24982.323367,34009.388051,11.386195,-433.133939,84.464639,-51.480614,15999.092811,0.053900,0.005219,0.001761,0.291046,-0.128461,0.144868,0.005487,-0.046597,0.842790,207.091316,117.756975,-19.903218,1.823059e+07,1.504187,-6447.494286,-19.591188,13488.294371,-4856.242874,-298.773863,1.917635,1.483075e+06,386778.797645,-158444.571203,-318998.318589,327478.592132,1.316769e+06,580084.265339,-1.309700e+06,0,192.168.10.19:62815
2,2.244337e+07,3.070306,16.580691,-8077.389277,51435.839651,-0.884177,39.926927,75.963219,-34.752774,155.694573,83.394822,-122.872561,-134.900733,-2.527749e+06,15028.431888,610413.511460,-6.285146e+05,-2.290991e+06,44517.885347,-5.570138e+06,-3.695320e+05,-6.446606e+05,4.229998e+05,-187334.786545,-2.782765e+06,4.335769e+05,1.244677e+06,1.572911e+06,-54162.457266,0.039617,0.002302,-1.311073e+07,2.092952e+06,-12331.494832,-33527.901085,52.409789,-623.552883,162.015173,192.089357,25141.614454,-0.044819,-0.015286,-0.041029,0.149858,-0.022269,-0.004487,-0.007778,-0.013033,0.754627,133.490097,101.821024,-71.050718,9.789191e+06,20.573170,1426.045697,-17.910390,34925.659513,3564.014192,-985.302618,-5.918761,-1.089829e+06,-296099.152102,274104.833144,-525136.766977,-338112.228410,-1.092818e+06,-359052.834848,7.972789e+05,0,192.168.10.5:61049
3,-2.239545e+07,0.929694,-12.580691,8221.389277,-51043.839651,72.884177,26.073073,-5.963219,34.752774,42.305427,82.605178,316.872561,134.900733,2.556058e+06,-14693.899815,-594490.178126,6.559665e+05,2.338811e+06,-44511.885347,5.570144e+06,3.695380e+05,6.446606e+05,-4.229938e+05,187340.786545,2.782773e+06,-4.335689e+05,-1.244677e+06,-1

In [28]:
# create dos profiles dataframe
num_hosts = len(host_dict)
num_src_hosts = len(activity_flows[' SrcHost'])
num_dos_hosts = num_noflow_hosts - num_low_hosts


no_flow_prof_columns = columns[3:-2]
print(no_flow_prof_columns)
dos_activities_prof = np.zeros((num_dos_hosts, len(no_flow_prof_columns)))
for col, attr in enumerate(no_flow_prof_columns):
  median = dos_activities[attr].median()
  std = np.std(dos_activities[attr])
  dos_activities_prof[:, col] = generate_random_array(median, std, num_dos_hosts)


dos_profile_df = pd.DataFrame(dos_activities_prof, columns = no_flow_prof_columns)
dos_profile_df[' Label'] = [2 for i in range(num_dos_hosts)]
dos_profile_df[' SrcHost'] = [hosts[num_src_hosts + num_low_hosts + i] for i in range(num_dos_hosts)]
dos_profile_df

[' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Fwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' ECE Flag Count', ' Down/Up R

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Label,SrcHost
0,5.950432e+07,4.951827,4.185964,279.143526,8326.278654,239.966707,0.347087,41.769643,116.089085,4223.963590,0.0,1330.236801,1695.851438,13090.214073,79044.172163,4.668585e+06,1.562252e+07,5.226073e+07,-8931.223807,6.114739e+07,9.898873e+06,2.379120e+07,5.860691e+07,13898.305457,1.359349e+07,2.735028e+06,6.500346e+06,1.190880e+07,232.742307,0.0,0.0,159.378013,146.082521,195775.908678,-12.826222,0.0,4036.337907,621.621772,1065.540319,2.250899e+06,0.270857,0.0,0.0,0.105755,0.613267,0.0,0.0,0.0,0.430067,595.097763,41.391545,1354.660443,183.970619,5.505616,267.537008,4.647317,8122.115245,2985.828630,189.689753,1.422212,27.620886,-45099.534513,0.0,-84986.373865,-43443.786012,5.819519e+07,8.998045e+05,5.886835e+07,2,192.168.10.19:35501
1,5.494554e+07,5.742291,4.876781,284.719219,7592.659255,264.856822,0.319579,48.730878,96.098850,3699.389351,0.0,1254.071089,1543.026879,6407.848249,230028.459964,5.006073e+06,1.707479e+07,5.982263e+07,8966.635572,5.281267e+07,1.068001e+07,2.341724e+07,5.347652e+07,-13617.505457,1.991310e+07,3.211362e+06,7.201375e+06,2.124724e+07,-84.456593,0.0,0.0,181.500419,155.439048,113271.336557,33.606393,0.0,3888.073858,578.513324,1373.656232,2.358295e+06,0.231103,0.0,0.0,0.123004,0.680850,0.0,0.0,0.0,0.393462,704.369481,49.108977,1229.647447,156.907813,5.188502,296.325737,4.415428,7796.822664,4018.880520,161.847502,1.138572,27.712447,49782.206782,0.0,89669.046134,48126.458281,5.309890e+07,-8.998045e+05,5.321398e+07,2,192.168.10.51:12477
2,5.277837e+07,5.099718,4.327270,286.812601,7781.490904,263.014092,0.461512,41.018870,110.596968,3466.645723,0.0,1354.304872,1558.213811,-8780.377088,115933.234273,4.996206e+06,1.585231e+07,5.136155e+07,-12024.727855,5.893266e+07,1.016414e+07,2.239371e+07,5.748154e+07,15233.387058,1.588525e+07,3.190757e+06,8.373432e+06,1.483781e+07,102.528505,0.0,0.0,158.481318,153.587927,116523.942711,1.074795,0.0,3540.826062,567.871506,1277.210661,2.347729e+06,0.272646,0.0,0.0,0.138023,0.608942,0.0,0.0,0.0,0.465594,647.632615,48.542179,1480.883590,180.208305,5.687398,291.797726,4.154641,8470.870406,3688.010525,188.329869,1.440615,27.111326,86939.639249,0.0,74826.490076,112472.330163,5.358858e+07,1.379485e+05,5.431608e+07,2,192.168.10.16:13860
3,6.167149e+07,5.594400,4.735475,277.050144,8137.447005,241.809437,0.205154,49.481652,101.590968,4456.707218,0.0,1230.003018,1680.664506,28278.439409,193139.397855,4.678451e+06,1.684500e+07,6.072181e+07,12060.139620,5.502739e+07,1.041474e+07,2.481473e+07,5.460189e+07,-14952.587058,1.762134e+07,2.755633e+06,5.328289e+06,1.831822e+07,45.757209,0.0,0.0,182.397114,147.933642,192523.302524,19.705377,0.0,4383.585703,632.263589,1161.985890,2.261465e+06,0.229315,0.0,0.0,0.090735,0.685175,0.0,0.0,0.0,0.357935,651.834629,41.958342,1103.424299,160.670127,5.006720,272.065019,4.908104,7448.067503,3316.698626,163.207386,1.120169,28.222007,-822

In [29]:
# combines all the dataframes
activity_flows_all = activity_flows.append(low_profile_df, ignore_index=True)
activity_flows_all = activity_flows_all.append(dos_profile_df, ignore_index=True)

print(set(activity_flows_all[' Label']))
non_atkers = activity_flows_all.groupby([' Label']).get_group(0)
ddos_atkers = activity_flows_all.groupby([' Label']).get_group(1)
dos_atkers = activity_flows_all.groupby([' Label']).get_group(2)
portscan_atkers = activity_flows_all.groupby([' Label']).get_group(3)

print("Number of non-atkers: " + str(len(non_atkers)))
print("Number of ddos " + str(len(ddos_atkers)))
print("Number of dos " + str(len(dos_atkers)))
print("Number of portscan " + str(len(portscan_atkers)))

activity_flows_all

{0, 1, 2, 3}
Number of non-atkers: 380953
Number of ddos 34216
Number of dos 6359
Number of portscan 14142


,SrcHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.0.0.0:0,3.668571e+07,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.054517,3.670000e+07,0.000000e+00,3.670000e+07,3.670000e+07,3.670000e+07,3.670000e+07,0.000000e+00,3.670000e+07,3.670000e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000,0.0,0.000000,0.000000,0.054517,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.670000e+07,0.000000e+00,3.670000e+07,36700000.0,0
1,1.1.70.73:80,4.511675e+04,2.000000,0.250000,3.000000,0.000000,1.500000,1.500000,1.500000,0.000000,0.000000,0.0,0.000000,0.000000,8.720930e+03,4244.011538,2.269288e+04,3.169376e+04,4.510375e+04,2.820000e+02,4.511675e+04,4.511675e+04,0.000000e+00,4.511675e+04,4.511675e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000,0.0,58.000000,8.000000,4242.617935,1.393604,1.5,1.500000,1.500000,0.000000,0.000000e+00,0.000000,0.000,0.0,0.000000,1.000000,0.25,0.0,0.0,0.000000,2.250000,1.500000,0.000000,58.000000,2.000000,3.000000,0.250000,0.000000,4454.250000,59.250000,0.250000,29.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0
2,1.193.219.21:443,6.480000e+02,3.000000,0.000000,258.000000,0.000000,258.000000,0.000000,86.000000,148.956369,0.000000,0.0,0.000000,0.000000,3.981481e+05,4629.629630,3.240000e+02,4.525483e+02,6.440000e+02,4.000000e+00,6.480000e+02,3.240000e+02,4.525483e+02,6.440000e+02,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000,0.0,96.000000,0.000000,4629.629630,0.000000,0.0,258.000000,64.500000,129.000000,1.664100e+04,0.000000,0.000,0.0,0.000000,1.000000,0.00,0.0,0.0,0.000000,86.000000,86.000000,0.000000,96.000000,3.000000,258.000000,0.000000,0.000000,61.000000,-1.000000,1.000000,32.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0
3,1.193.219.24:443,1.038322e+05,2.000000,0.500000,258.000000,0.000000,258.000000,0.000000,129.000000,182.433549,0.000000,0.0,0.000000,0.000000,5.376194e+07,416805.849036,2.604478e+04,5.196309e+04,1.038094e+05,1.187500e+01,1.038251e+05,1.038251e+05,0.000000e+00,1.038251e+05,1.038251e+05,1.034842e+05,5.174212e+04,7.317388e+04,1.034839e+05,0.375000,0.875,0.0,64.000000,10.000000,416759.253610,46.595426,0.0,258.000000,150.500000,143.502831,2.080125e+04,0.000000,0.875,0.0,0.000000,1.000000,0.00,0.0,0.0,0.125000,221.450000,129.000000,0.000000,64.000000,2.000000,258.000000,0.500000,0.000000,65.000000,-0.750000,0.125000,32.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0
4,1.226.51.14:80,4.217540e+04,1.400000,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [30]:
### output to the dataframe to csv file for evaluation with other models
activity_flows_all.to_csv('hosts_multclass.csv')

In [ ]:
# reformat Source IP
activity_flows_all[" SrcHost"] = activity_flows_all[" SrcHost"].map(host_dict)
print(activity_flows_all)


         SrcHost   Flow Duration  ...  group_size   Label
0          45751    3.668571e+07  ...         1.0       0
1         114204    4.511675e+04  ...         4.0       0
2         178773    6.480000e+02  ...         1.0       0
3           1723    1.038322e+05  ...         5.0       0
4         388757    4.217540e+04  ...         5.0       0
...          ...             ...  ...         ...     ...
435665    435665    5.214953e+07  ...         NaN       2
435666    435666    6.230033e+07  ...         NaN       2
435667    435667    5.382325e+07  ...         NaN       2
435668    435668    6.062661e+07  ...         NaN       2
435669    435669    5.190492e+07  ...         NaN       2

[435670 rows x 72 columns]


In [ ]:
activity_flows_all.to_csv("ip:port_multiclass.csv")

In [ ]:
activity_flows_all = activity_flows_all.sort_values(by=' SrcHost', ignore_index=True)
activity_flows_all

,SrcHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,group_size,Label
0,0,5.821268e+07,15.250000,15.500000,4117.000000,5731.500000,1407.000000,0.000000,206.490556,464.609045,1835.500000,0.0,315.479167,504.398230,4.420796e+06,166674.482109,1.217419e+06,2.140721e+06,5.054842e+06,2.500000,5.821263e+07,2.441276e+06,2.525287e+06,5.077818e+06,50.750000,5.554624e+07,2.419850e+06,2.580687e+06,5.078831e+06,100.250000,0.25,0.0,494.000000,502.000000,166669.749985,4.732123,0.0,1848.750000,272.233750,495.901669,3.193793e+05,0.000000,0.25,0.0,0.750000,0.250000,0.0,0.0,0.0,0.250000,284.984916,206.490556,315.479167,494.000000,15.250000,4117.000000,15.500000,5731.500000,34208.000000,1857.000000,4.250000,32.000000,33882.541667,45184.018784,1.773610e+05,20744.500000,4.801421e+06,6.933232e+05,5.009110e+06,2.599938e+06,3.0,0
1,1,1.673435e+05,3.000000,2.250000,295.550000,213.050000,156.900000,21.400000,72.483333,65.186895,134.900000,40.9,67.150000,40.522370,3.039703e+05,5234.801694,4.310793e+04,6.531845e+04,1.282745e+05,41.500000,1.492914e+05,5.176691e+04,7.238088e+04,1.274969e+05,1467.600000,9.201345e+04,2.300801e+04,3.154631e+04,6.972320e+04,426.950000,0.00,0.0,65.400000,50.400000,2619.023620,2615.778075,21.4,177.900000,65.567045,63.939167,1.018245e+04,0.000000,0.00,0.0,0.250000,0.250000,0.0,0.0,0.0,0.750000,77.083333,72.483333,67.150000,65.400000,3.000000,295.550000,2.250000,213.050000,2111.000000,32.000000,2.000000,21.200000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.0,0
2,2,1.670000e+02,2.000000,2.000000,86.000000,204.000000,43.000000,43.000000,43.000000,0.000000,102.000000,102.0,102.000000,0.000000,1.736527e+06,23952.095810,5.566667e+01,5.589574e+01,1.150000e+02,4.000000,4.000000e+00,4.000000e+00,0.000000e+00,4.000000e+00,4.000000,4.800000e+01,4.800000e+01,0.000000e+00,4.800000e+01,48.000000,0.00,0.0,64.000000,64.000000,11976.047900,11976.047900,43.0,102.000000,66.600000,32.315631,1.044300e+03,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,1.000000,83.250000,43.000000,102.000000,64.000000,2.000000,86.000000,2.000000,204.000000,-1.000000,-1.000000,1.000000,32.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0
3,3,3.584850e+04,3.000000,2.500000,120.500000,256.500000,44.000000,44.000000,44.000000,0.000000,112.500000,112.5,112.500000,0.000000,9.173242e+05,9888.185739,6.646433e+03,1.244880e+04,2.980375e+04,14.500000,1.373025e+04,4.585750e+03,7.906818e+03,1.371575e+04,15.250000,1.672500e+03,5.740000e+02,9.398709e+02,1.659250e+03,25.750000,0.00,0.0,78.000000,62.000000,4946.464967,4941.720774,44.0,112.500000,68.796825,35.850062,1.523930e+03,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.750000,83.281250,44.000000,112.500000,78.000000,3.000000,120.500000,2.500000,256.500000,-1.000000,-1.000000,2.000000,23.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.

In [ ]:
# (1) SELECT | ALL features
X = activity_flows_all.iloc[:, 1:(activity_flows_all.shape[1]-1)]
Y = activity_flows_all.iloc[:, -1]


In [ ]:
# create hypergraph
# connection = traffic.groupby([' SrcHost', ' DstHost'], as_index=False)
# connection = connection.agg({' Flow Duration' : 'count'})
src_host_ids = [host_dict[host] for host in basic_flows[' SrcHost']]
dst_host_ids = [host_dict[host] for host in basic_flows[' DstHost']]
edges = list(zip(src_host_ids, dst_host_ids))
hypergraph = {i: edges[i] for i in range(len(edges))}

In [ ]:
# # generate features
features = X
features = np.nan_to_num(features)


# generate features from feature selection set
# X = X.fillna(0)
# selected_features = selectFeatures(X, Y)
# print(f'Selected Features "from All": {selected_features}')
# features = X[selected_features]
# print("Shape of features: ", features.shape)

# standardisation
scaler = MinMaxScaler()
features = scaler.fit_transform(features)
print(features)
features = csr_matrix(features)



In [ ]:
# generate labels
labels = Y


In [ ]:
# train/test split

splits = {}
V = len(hosts)
node_sorted = list(range(V))
rnd.shuffle(node_sorted)
train_size = round(0.8*V)
test_size = V - train_size
train_set, test_set = train_test_split(node_sorted, train_size=train_size, test_size=test_size)



splits['train'] = train_set
splits['test'] = test_set


In [ ]:
train_atk_count = 0
for idx, train_id in enumerate(train_set):
    if labels[train_id] !=  0:
      train_atk_count += 1

test_atk_count = 0
for idx, test_id in enumerate(test_set):
    if labels[test_id] != 0:
      test_atk_count += 1

print("Number of atker in train set: ", train_atk_count)
print("Number of atker in test set: ", test_atk_count)

Number of atker in train set:  43728
Number of atker in test set:  10989


In [ ]:
# generate files
with open("hypergraph.pickle", "wb") as hypergraph_out:
    pickle.dump(hypergraph, hypergraph_out)
    hypergraph_out.close()
with open("features.pickle", "wb") as features_out:
    pickle.dump(features, features_out)
    features_out.close()
with open("labels.pickle", "wb") as labels_out:
    pickle.dump(labels, labels_out)
    labels_out.close()
with open("1.pickle", "wb") as splits_out:
    pickle.dump(splits, splits_out)
    splits_out.close()